Written by Marcin "MrZiemni0k".

My 1st WebScraping Project. Still needs some improvement but at least it works.
Planning to add:
* Interactions between an user and the script:
    - Check for dead hosts by .netloc?
    - Delete .netlock urls? 
    - Store timed out urls? 
    - Directory input?
    - Minimum size of the pictures to be downloaded?
* Compress code into nicer version. Add classes.
* Make nested dictionary: {Sub page:{Thread:[Urls]}} for smoother interaction with an user. For example to let's say to avoid sub pages that are not interesting for us. 
* Adjust url patterns for web-crawlers.

A script is towards common public forum which is 3-page deep(Main Page, Sub Page(Categories) and Threads with a content 
we are looking for. My script's main goal is to download all images(with some adjustments) from urls hidden within bbcode.
                                                             
My target was a medium size japanese archive forum (12years of content). It took around 2 days to webscrape urls 
(Could be done faster depending on set value of time.sleep())
and 4 days to download all images after some sorting and cleaning - Many hosts were slow or dead etc. 

In [ ]:
import requests
import bs4
import re
from urllib.error import HTTPError
from urllib.error import URLError
import time

def geturlThreadPage(pageUrl):
#   Function to store thread urls in sublist.
#   Passed argument will be url for subpage stored in mainlist.
#   Pattern of urls in subpages is: .../{mainlist}-page-{number}.html
    
    num_page = 1                  
    while True: 
        
        try:                    
            website = 'http://www.somekindofwebsite.jp//{}-page-{}.html'.format(pageUrl[:-5],num_page)  
                                                                    #Slicing {mainlist} to put "-page-{number}"
            readweb = requests.get(website)       
            #time.sleep()                           #OPTIONAL! - Giving a rest for a server - Good, gut bot :3
        except HTTPError: 
            return("HTTPError")
        except URLError:
            return('The server could not be found!')
        except:
            return("No Connection")
             
             
        bs = bs4.BeautifulSoup(readweb.text,"lxml")        
        looplist = []                           #Empty list for breaking the loop if url is not found on empty page. 
                                                #Let's say subpage count is 50, so on 51st page I want the loop to stop.
        
        for url in bs.find_all("li"):                     #Urls are stored in html lists - looking there.
            if url.a != None:                             #If urls stored at "a href=" withing tables:
                sublist.append(url.a["href"])             #Add urls to sublist for the next func to work with later.
                looplist.append(url.a["href"])            #Add urls to looplist to chck later if any urls found within bs.find_all.
        if len(looplist) == 0:                            #If looplist is empty/no stored urls then most probably we chcecked all pgs within subpage.
            print("Breaking loop at {}".format(website))   #Giving info to check progress.
            time.sleep(0.2)                                 #OPTIONAL! Being gud, gut bot :3
            break
        else:
            num_page += 1                                        #Urls found make num_page+1


            
def geturlImgPage(urlsub,imglist):
#Function to store image_urls in imglist list and file.
#Passed arguments are:
#urlsub for thread stored in sublist
#a list to store our imgurls
#Pattern:  .../{sublist}-page-{number}.html


    def should_I_look_img():
        '''
        Assist Function to check 1st page of a thread for urls using regular expressions defined in specify.
        '''
        for img in anytext:                                  
            findurl=re.search(specify,img.text)
            if findurl:
                return True
        return False
    
    
    def should_I_look_txt(): 
        '''
        Assist Function to check if we are on the last page of a thread.
        '''
        for img in anytext:                                  
            findurl=re.search("\S+",img.text)
            if findurl:
                return True
        return False
    
    specify = re.compile("\[img\]+\S+\[/img\]", re.IGNORECASE)  #Filter to find [img](anything)[/img].lower()
    chck_loop_url = 0                                           #Loop to skip no good content threads.
    num_page = 1            
    while True:       
        try:       
            
            website = 'http://www.somekindofwebsite.jp//{}-page-{}.html'.format(urlsub[:-5],num_page)  
                                                                #Slicing {sublist} to put "-page-{number}"
            readweb = requests.get(website)          
            #time.sleep()                           #OPTIONAL! Giving a rest for a server - Good, gut bot :3
        except HTTPError:
            return("HTTPError")
        except URLError:
            return('The server could not be found!')
        except:
            return("No Connection")
        
        bs = bs4.BeautifulSoup(readweb.text,"lxml")                       
        anytext = bs.find_all("div", {"class":"subtable altbg2 t_msg"})  
        #Mesages (in our case - The imgurls) are hidden within broken bbcode in <div> "subtable altbg2 t_msg" - class.
        
        store_result = should_I_look_img()
        
        if num_page == 1 and store_result: #Chck for 1st page if any images found..                                   
                                                         #If not then most probably no images at all in a thread. 
                                                         #Skip to not waste the time.              
            for img in anytext:  
                findurl=re.findall(specify,img.text)
                for url in findurl:
                    print(f'Adding: {url[5:-6]}')
                    imglist.append(url[5:-6])
                    f.write(url[5:-6] + '\n')
                    
                    
        elif num_page!=1 and should_I_look_txt():        
                                                         #Chck if 3 pages in a row are without pictures.
                                                         #Skip to not waste time. Some threads were over 1000pages.
                                                         #Less probability to find anything useful on later pages.
                    
            if chck_loop_url == 2:
                print("-!-!-!-!-!-!-!")
                print("Too many pages without image urls. Going next thread.")
                chck_loop_url = 0
                del sublist[0]
                break
            
            elif store_result:
            
                
                for img in anytext:                                  
                    findurl=re.findall(specify,img.text)
                    for url in findurl:
                        print(f'Adding: {url[5:-6]}')
                        imglist.append(url[5:-6])
                        chck_loop_url = 0
                        f.write(url[5:-6] + '\n')
                        
            
            else:
                chck_loop_url += 1
               
                                                                
        elif num_page == 1:                                             
            print("No Imgs Found on 1st Page. Going next thread.")
            del sublist[0]
            break
        else:                                              
            print("No Page Found. Breaking loop at: {}".format(website))         
            del sublist[0]
            break
            
        num_page += 1                                        
        
        
    
#//////////////////////////////////////////////////////////////////////////////////////////
#                                Some lists/arguments 
#//////////////////////////////////////////////////////////////////////////////////////////
mainlist = [] #Store subpages urls
sublist = []  #Store threads urls
imglist = []  #Store img urls
f = open("Zdjecia.txt", "w", encoding="utf-8")
#//////////////////////////////////////////////////////////////////////////////////////////
#                    Main page - looking for subpages (No func)
#//////////////////////////////////////////////////////////////////////////////////////////
#Checking main webpage for subpages. And adding them into mainlist.

try:
    website = requests.get('http://www.somekindofwebsite.jp/archiver/')
except HTTPError:
    print("HTTPError")
except URLError:
    print('The server could not be found!')
except:
    print("No Connection")
        
soup = bs4.BeautifulSoup(website.text,"lxml")
for url in soup.find_all("li"):
    if url.a != None:
        mainlist.append(url.a["href"])

print(mainlist)  

#//////////////////////////////////////////////////////////////////////////////////////////
#                   Executing funcs and storing imgurl list in new file
#//////////////////////////////////////////////////////////////////////////////////////////

for x in mainlist:
    geturlThreadPage(x)


for x in sublist:
    geturlImgPage(x,imglist)
    
f.close()

#with open("imglist.txt", "w",encoding="utf-8") as output: 
    #output.write(str(imglist))

Output Example:
    
['archiver/?pid-42.html', 'archiver/?pid-05.html', 'archiver/?pid-96.html', 'archiver/?pid-86.html', 'archiver/?pid-73.html', 'archiver/?pid-69.html', 'archiver/?pid-50.html', 'archiver/?pid-46.html', 'archiver/?pid-31.html', 'archiver/?pid-25.html', 'archiver/?pid-15.html', 'archiver/?pid-91.html', 'archiver/?pid-81.html', 'archiver/?pid-57.html', 'archiver/?pid-65.html', 'archiver/?pid-54.html', 'archiver/?pid-42.html', 'archiver/?pid-33.html', 'archiver/?pid-27.html', 'archiver/?pid-19.html']

Breaking loop athttp://www.somekindofwebsite.jp//archiver/?pid-42-page-3.html

No Imgs Found on 1st Page. Going next thread.

Adding: https://www.someurlhost.com/2020/08/04/fbeuifu.jpg

Adding: https://www.someurlhost.com/2020/08/04/dwqfewe.jpg

Adding: https://www.someurlhost.com/2020/08/04/gerrgege8.jpg

Adding: https://www.someurlhost.com/2020/08/04/ewqeqweqw.jpg

Adding: https://www.someurlhost.com/2020/08/04/fewgregeg.jpg

No Page Found. Breaking loop at: http://www.somekindofwebsite.jp//archiver/?pid-34553-page-2.html

No Imgs Found on 1st Page. Going next thread.

No Imgs Found on 1st Page. Going next thread.

No Imgs Found on 1st Page. Going next thread.

No Imgs Found on 1st Page. Going next thread.

No Imgs Found on 1st Page. Going next thread.
         
        

Script is not fully automated yet. Still I need to sort and clean urls manually.
I have now a file with all urls (some dead urls, some hosts might be very slow). First I would like to clean. 
Below cell is proposition how to organize urls - It's just an option. 


In [ ]:
from urllib.parse import urlparse
from collections import Counter

photos = open("zdjecia2.txt", "r",encoding = "utf-8")
imglist = photos.read().split("\n")

#print(f'Amount of urls: {len(imglist)}')
imglist = set(imglist)         #Eliminate all repeative urls.
imglist = list(imglist)
#print(f'Amount of urls: {len(imglist)}')

netloclist = []                #List to store all netloc urls. 
for url in imglist:
    link = urlparse(url)
    netloclist.append(link.netloc)

cnt = Counter()
for url in netloclist:
    cnt[url] += 1
dict(sorted(cnt.items(), key=lambda item: item[1], reverse = True)) #Found on Stack overflow
                                                                    #https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value

Output example:

Amount of urls: 515550

Amount of urls: 453813

{'imgur.net': 115851,

 'castles.com': 61228,
 
 'pics.jp': 17881,
 
 'img070.com': 13310,
 
 'jpnet.com': 11872,
 
 ...} etc.
 

I am planning to add to the script list of urls with long connection, errors and timeouts. Every 500-1000(user input) checked 
urls and depending on a decision from the user -> use function to delete all urls with not convenient urls/continue or never ask again.

For now netlocks with big numbers I check manually 10 examples. If everything is fine I go to the next one until small numbers occur like let's say 5k.

In [ ]:
import requests
import bs4
import re
from urllib.error import HTTPError
from urllib.error import URLError
from urllib.parse import urlparse
import time
import urllib
import os
import sys
from collections import Counter

#//////////////////////////////////////////////////////////////////////////////////////////
#                                Some lists/variables
#//////////////////////////////////////////////////////////////////////////////////////////

counturl = 0                  #Counters to get updated on status.
imgstored = 0
sizetotal = 0
sizestored = 0
# avoidlinks = ["somewebsite.jpg",'iamslow.png']   # Beginner way of manually avoiding urls. Not needed anymore - For future learning not deleted yet.
directory = "E:\ScrapedImgs\_"
chckforurl = "" #Variable to store chcked url. If host was slow - not waste time - set timeout to quick one. 
                #Else 5 secs - Some images might be big and take time to download.

def download_img(url,directory):    
#Function taking url - connecting to website and then storing image into specified directory.
#I will be working to make it into class.
    
    # global avoidlinks
    global counturl
    global imgstored
    global sizetotal
    global sizestored
    global chckforurl
    a = urlparse(url)
    b = "".join(a.path.split("/")) #For the name of picture file.
    counturl += 1
    if a.netloc == chckforurl:
        timeval = 0.2
    else:
        timeval = 5
    try:
        response = requests.get(url,allow_redirects = False, stream = True, timeout = timeval)     
    except HTTPError: 
        return("HTTPError")
    except URLError:
        return('The server could not be found!')
    except requests.exceptions.Timeout:
        print(url)
        chckforurl = a.netloc
        return("Timeout occurred")            
    except ConnectionError:  
        return("ConnectionError")      
    except:
        print(f"Could not retrieve image! Image {a.path} not added.")
        return None
    if "[/img]" in a.path:              #Bad formating from users in BB code - Couple of urls might be messed up.
        return None
         
    
    if '?' in b or "*" in b or ":" in b:  #Not accepted symbols for creating file names.
        b = b[:b.find('?')]
        b = b[:b.find("*")]
        b = b[:b.find(":")]
        
        
    if "." in a.path[-5:-3]:
        symbol = True
        file = open(f"{directory}{b}", "wb")
        file.write(response.content)
        file.close()
        size =((os.stat(f"{directory}{b}").st_size)/1024)
        sizetotal += (size/1024)
    
    
    else: 
        symbol = False
        file = open(f"{directory}{b}.jpg", "wb")
        file.write(response.content)
        file.close()
        size =((os.stat(f"{directory}{b}.jpg").st_size)/1024)
        sizetotal += (size/1024)
       
    if size < 60:
        if symbol:
            os.remove(f"{directory}{b}")
    
               
        else:
            try:
                os.remove(f"{directory}{b}.jpg")
            except:
                print("Sth went wrong!")
           
              
    else:
        sizestored += (size/1024)
        imgstored += 1
        print("Creating file: ...{} // Size: {} KB // Imgs_Stored: {} // Urls_chcked {} //Size stored: {} MB /Total: {} MB".format(b[-8:],round(size,2),imgstored,counturl,round(sizestored,2),round(sizetotal,2)))
 
        
        
    

        

In [ ]:
for x in imglist:
    try:
        download_img(x,"E:\ScrapedImgs\_")
    except:
        print("Sth is wrong") 


Output example:

Could not retrieve image! Image /api/cmfs/3/543/ogp/120.jpg not added.

Could not retrieve image! Image  not added.

https://www.somekind/blehiwe4.jpg

Timeout occurred

Creating file: ...jp-1.jpg // Size: 70.23 KB // Imgs_Stored: 55100 // Urls_chcked 244372 //Size stored: 10093.35 MB /Total: 11211.59 MB
